### Imports 

In [37]:
import requests
import json
from bs4 import BeautifulSoup
import os

### Extraction of opinions


In [38]:
with open('./cookie.json','r', encoding="UTF-8") as jf:
    headers = json.load(jf)

In [43]:
product_id = "17230358"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        
        for opinion in opinions:
            single_opinion = {}
            single_opinion["opinion_id"] = opinion["data-entry-id"]
            single_opinion["author"] = opinion.select_one("span.user-post__author-name").text
            try:
                single_opinion["recommend"] = opinion.select_one("span.user-post_author-recomendation > em").text
            except AttributeError:
                single_opinion["recommend"] = None

            try:
                single_opinion["stars"] = opinion.select_one("span.user-post__score-count").text
            except AttributeError:
                single_opinion["stars"] = None

            try:
                single_opinion["content"] = opinion.select_one("div.user-post__text").text

            except AttributeError:
                single_opinion["content"] = None

            single_opinion["pros"] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--positive")]
            single_opinion["cons"] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--negative")]

            single_opinion["up_votes"] = opinion.select_one("button.vote-yes")["data-total-vote"]
            single_opinion["down_votes"] = opinion.select_one("button.vote-no")["data-total-vote"]
            single_opinion["published"] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
            try:
                single_opinion["purchased"] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
            except TypeError:
                single_opinion["purchased"] = None

            all_opinions.append(single_opinion)

        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None
        
    
    

In [44]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [45]:
with open(f"./opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [46]:
product_id = "17230358"